![DSME-logo](./img/DSME_logo.png)

#  Reinforcement Learning and Learning-based Control

<p style="font-size:12pt";> 
<b> Prof. Dr. Sebastian Trimpe, Dr. Friedrich Solowjow </b><br>
<b> Institute for Data Science in Mechanical Engineering (DSME) </b><br>
<a href = "mailto:rllbc@dsme.rwth-aachen.de">rllbc@dsme.rwth-aachen.de</a><br>
</p>

---

# Linear Value Function Approximation
*Code adapted from: https://github.com/ShangtongZhang/reinforcement-learning-an-introduction*

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Random Walk Example
<center><img width="800" src=./img/random_walk.png /></center>

**Example 9.1 in Sutton & Barto:**
Consider a
1000-state version of the random walk task (Examples 6.2 and 7.1 on pages 125 and
144). The states are numbered from 1 to 1000, left to right, and all episodes begin near
the center, in state 500. State transitions are from the current state to one of the 100
neighboring states to its left, or to one of the 100 neighboring states to its right, all with
equal probability. Of course, if the current state is near an edge, then there may be fewer
than 100 neighbors on that side of it. In this case, all the probability that would have
gone into those missing neighbors goes into the probability of terminating on that side
(thus, state 1 has a 0.5 chance of terminating on the left, and state 950 has a 0.25 chance
of terminating on the right). As usual, termination on the left produces a reward of -1
, and termination on the right produces a reward of +1. All other transitions have a
reward of zero. We use this task as a running example throughout this section


Environment Hyperparameters

In [ ]:
# set rewards and temporal discounting
REWARD_LEFT = -1
REWARD_RIGHT = 1
REWARD_BETWEEN = 0

GAMMA = 1

# # of states except for terminal states
N_STATES = 1000
# all states
STATES = np.arange(1, N_STATES + 1)
# start from a central state
START_STATE = 500
# terminal states
END_STATES = [0, N_STATES + 1]

# possible actions
ACTION_LEFT = -1
ACTION_RIGHT = 1
ACTIONS = [ACTION_LEFT, ACTION_RIGHT]
# maximum stride for an action
STEP_RANGE = 100


## Tabular Dynamic Programming to compute 'true' Value Function

In [ ]:
def compute_true_value():

    true_value = np.zeros((N_STATES + 2))

    # Dynamic programming to find the true state values
    while True:
        old_value = np.copy(true_value)
        for state in STATES:
            true_value[state] = 0
            for action in ACTIONS:
                for step in range(1, STEP_RANGE + 1):
                    step *= action
                    next_state = state + step
                    next_state = max(min(next_state, N_STATES + 1), 0)
                    # asynchronous update for faster convergence
                    if next_state == 0:
                        true_value[state] += 1.0 / (2 * STEP_RANGE) * (REWARD_LEFT)
                    elif next_state == N_STATES + 1:
                        true_value[state] += 1.0 / (2 * STEP_RANGE) * (REWARD_RIGHT)
                    else:
                        true_value[state] += 1.0 / (2 * STEP_RANGE) * (REWARD_BETWEEN + GAMMA * true_value[next_state])

        error = np.sum(np.abs(old_value - true_value))
        print('The remaining error is: ' + str(np.round(error, 3)))
        if error < 1e-2:
            break
    # correct the state value for terminal states to 0
    true_value[0] = true_value[-1] = 0

    return true_value

## Random Walk Environment

In [ ]:
# take an @action at @state, return new state and reward for this transition
def step(state, action):
    step = np.random.randint(1, STEP_RANGE + 1)
    step *= action
    state += step
    state = max(min(state, N_STATES + 1), 0)
    if state == 0:
        reward = REWARD_LEFT
    elif state == N_STATES + 1:
        reward = REWARD_RIGHT
    else:
        reward = REWARD_BETWEEN
    return state, reward

## Random Policy

In [ ]:
# get an action, following random policy
def get_action():
    if np.random.binomial(1, 0.5) == 1:
        return 1
    return -1

## Different Types of Value Functions

In [ ]:
# a wrapper class for aggregation value function
class ValueFunction:
    # @num_of_groups: # of aggregations
    def __init__(self, num_of_groups):
        self.num_of_groups = num_of_groups
        self.group_size = N_STATES // num_of_groups

        # thetas
        self.params = np.zeros(num_of_groups)

    # get the value of @state
    def value(self, state):
        if state in END_STATES:
            return 0
        group_index = (state - 1) // self.group_size
        return self.params[group_index]

    # update parameters
    # @delta: step size * (target - old estimation)
    # @state: state of current sample
    def update(self, delta, state):
        group_index = (state - 1) // self.group_size
        self.params[group_index] += delta

# a wrapper class for tile coding value function
class TilingsValueFunction:
    # @num_of_tilings: # of tilings
    # @tileWidth: each tiling has several tiles, this parameter specifies the width of each tile
    # @tilingOffset: specifies how tilings are put together
    def __init__(self, numOfTilings, tileWidth, tilingOffset):
        self.numOfTilings = numOfTilings
        self.tileWidth = tileWidth
        self.tilingOffset = tilingOffset

        # To make sure that each sate is covered by same number of tiles,
        # we need one more tile for each tiling
        self.tilingSize = N_STATES // tileWidth + 1

        # weight for each tile
        self.params = np.zeros((self.numOfTilings, self.tilingSize))

        # For performance, only track the starting position for each tiling
        # As we have one more tile for each tiling, the starting position will be negative
        self.tilings = np.arange(-tileWidth + 1, 0, tilingOffset)

    # get the value of @state
    def value(self, state):
        stateValue = 0.0
        # go through all the tilings
        for tilingIndex in range(0, len(self.tilings)):
            # find the active tile in current tiling
            tileIndex = (state - self.tilings[tilingIndex]) // self.tileWidth
            stateValue += self.params[tilingIndex, tileIndex]
        return stateValue

    # update parameters
    # @delta: step size * (target - old estimation)
    # @state: state of current sample
    def update(self, delta, state):

        # each state is covered by same number of tilings
        # so the delta should be divided equally into each tiling (tile)
        delta /= self.numOfTilings

        # go through all the tilings
        for tilingIndex in range(0, len(self.tilings)):
            # find the active tile in current tiling
            tileIndex = (state - self.tilings[tilingIndex]) // self.tileWidth
            self.params[tilingIndex, tileIndex] += delta

# a wrapper class for polynomial / Fourier -based value function
POLYNOMIAL_BASES = 0
FOURIER_BASES = 1
class BasesValueFunction:
    # @order: # of bases, each function also has one more constant parameter (called bias in machine learning)
    # @type: polynomial bases or Fourier bases
    def __init__(self, order, type):
        self.order = order
        self.weights = np.zeros(order + 1)

        # set up bases function
        self.bases = []
        if type == POLYNOMIAL_BASES:
            for i in range(0, order + 1):
                self.bases.append(lambda s, i=i: pow(s, i))
        elif type == FOURIER_BASES:
            for i in range(0, order + 1):
                self.bases.append(lambda s, i=i: np.cos(i * np.pi * s))

    # get the value of @state
    def value(self, state):
        # map the state space into [0, 1]
        state /= float(N_STATES)
        # get the feature vector
        feature = np.asarray([func(state) for func in self.bases])
        return np.dot(self.weights, feature)

    def update(self, delta, state):
        # map the state space into [0, 1]
        state /= float(N_STATES)
        # get derivative value
        derivative_value = np.asarray([func(state) for func in self.bases])
        self.weights += delta * derivative_value

class PolynomialValueFunction:
    def __init__(self, order):
        self.order = order
        self.weights = np.zeros(order +1)

    def featurize(self, state):
        feature = np.zeros(self.order + 1)
        state /= float(N_STATES)
        for i in range(self.order +1):
            feature[i] = state ** i
        return feature

    def value(self, state):
        return np.dot(self.featurize(state), self.weights)

    def update(self, delta, state):
        grad_v = self.featurize(state)
        self.weights += delta * grad_v


## Gradient Monte-Carlo Algorithm 
(Sutton & Barto page 202)

In [ ]:
def gradient_monte_carlo(value_function, alpha, distribution=None):
    state = START_STATE
    trajectory = np.array([[state, 0. ]])

    # We assume gamma = 1, so return is just the same as the latest reward
    #reward = 0.0
    while state not in END_STATES:
        action = get_action()
        next_state, reward = step(state, action)
        trajectory = np.append(trajectory, np.array([[next_state, reward]]), axis=0)
        state = next_state

    # Gradient update for each state in this trajectory

    ret = 0.0
    traj_len = trajectory.shape[0]
    for i in range(traj_len-1):
        state = int(trajectory[traj_len - 2 - i, 0])
        reward = int(trajectory[traj_len -1 - i, 1])
        ret = reward + GAMMA*ret
        delta = alpha * (ret - value_function.value(state))
        value_function.update(delta, state)
        if distribution is not None:
            distribution[state] += 1
        

## Semi-gradient TD(0) Algorithm
(Sutton & Barto page 203)

In [ ]:
def semi_gradient_td_0(value_function, alpha, distribution=None):
    state = START_STATE

    while not state in END_STATES:
        action = get_action()
        next_state, reward = step(state, action)
        if next_state == 0:
            delta = alpha*(reward  - value_function.value(state))
        elif next_state == N_STATES+ 1:
            delta = alpha*(reward  - value_function.value(state))
        else:
            delta = alpha*(reward + GAMMA * value_function.value(next_state) - value_function.value(state))
        value_function.update(delta, state)
        if distribution is not None:
            distribution[state] += 1
        state = next_state

### Compute true state value function using tabular dynamic programming

In [ ]:
true_value = compute_true_value()

### MC learning with state aggregation feature

In [ ]:
episodes = int(1e5)
alpha = 5e-5

# we have 10 aggregations in this example, each has 100 states
value_function = ValueFunction(10)

distribution = np.zeros(N_STATES + 2)
for ep in tqdm(range(episodes)):
    gradient_monte_carlo(value_function, alpha, distribution)
    
distribution /= np.sum(distribution)
state_values = [value_function.value(i) for i in STATES]

plt.figure(figsize=(10, 20))

plt.subplot(2, 1, 1)
plt.plot(STATES, state_values, label='Approximate MC value')
plt.plot(STATES, true_value[1: -1], label='True value')
plt.xlabel('State')
plt.ylabel('Value')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(STATES, distribution[1: -1], label='State distribution')
plt.xlabel('State')
plt.ylabel('Distribution')
plt.legend()
plt.show()

### TD Learning with state aggregation feature

In [ ]:
episodes = int(1e5)
alpha = 5e-5

# we have 10 aggregations in this example, each has 100 states
value_function = ValueFunction(10)

distribution = np.zeros(N_STATES + 2)
for ep in tqdm(range(episodes)):
    semi_gradient_td_0(value_function, alpha, distribution)
    
distribution /= np.sum(distribution)
state_values = [value_function.value(i) for i in STATES]

plt.figure(figsize=(10, 20))

plt.subplot(2, 1, 1)
plt.plot(STATES, state_values, label='Approximate TD value')
plt.plot(STATES, true_value[1: -1], label='True value')
plt.xlabel('State')
plt.ylabel('Value')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(STATES, distribution[1: -1], label='State distribution')
plt.xlabel('State')
plt.ylabel('Distribution')
plt.legend()
plt.show()

### MC learning with polynomial feature (affine function)

In [ ]:
episodes = int(5e5)
alpha = 2e-5

# polynomial of order 1
value_function = BasesValueFunction(1, POLYNOMIAL_BASES)

distribution = np.zeros(N_STATES + 2)
for ep in tqdm(range(episodes)):
    gradient_monte_carlo(value_function, alpha, distribution)
    
distribution /= np.sum(distribution)
state_values = [value_function.value(i) for i in STATES]

plt.figure(figsize=(10, 10))

plt.plot(STATES, state_values, label='Approximate MC value')
plt.plot(STATES, true_value[1: -1], label='True value')
plt.xlabel('State')
plt.ylabel('Value')
plt.legend()
plt.show()

### TD learning with polynomial feature (affine function)

In [ ]:
episodes = int(5e5)
alpha = 2e-5

# polynomial of order 1
value_function = BasesValueFunction(1, POLYNOMIAL_BASES)

distribution = np.zeros(N_STATES + 2)
for ep in tqdm(range(episodes)):
    semi_gradient_td_0(value_function, alpha, distribution)
    
distribution /= np.sum(distribution)
state_values = [value_function.value(i) for i in STATES]

plt.figure(figsize=(10, 10))

plt.plot(STATES, state_values, label='Approximate TD value')
plt.plot(STATES, true_value[1: -1], label='True value')
plt.xlabel('State')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
GAMMA = 0.5
true_value = compute_true_value()
episodes = int(5e5)
alpha = 2e-5

# polynomial of order 1
value_function = BasesValueFunction(1, POLYNOMIAL_BASES)

distribution = np.zeros(N_STATES + 2)
for ep in tqdm(range(episodes)):
    semi_gradient_td_0(value_function, alpha, distribution)
    
distribution /= np.sum(distribution)
state_values = [value_function.value(i) for i in STATES]

plt.figure(figsize=(10, 10))

plt.plot(STATES, state_values, label='Approximate TD value')
plt.plot(STATES, true_value[1: -1], label='True value')
plt.xlabel('State')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
episodes = int(1e6)
alpha = 1e-4

# polynomial of order 5
value_function = BasesValueFunction(5, POLYNOMIAL_BASES)

distribution = np.zeros(N_STATES + 2)
for ep in tqdm(range(episodes)):
    semi_gradient_td_0(value_function, alpha, distribution)
    
distribution /= np.sum(distribution)
state_values = [value_function.value(i) for i in STATES]

plt.figure(figsize=(10, 10))

plt.plot(STATES, state_values, label='Approximate TD value')
plt.plot(STATES, true_value[1: -1], label='True value')
plt.xlabel('State')
plt.ylabel('Value')
plt.legend()
plt.show()